In [2]:
# python required >= 3.5 version
import sys
assert sys.version_info >=(3,5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

import numpy as np
import os
import pandas as pd
# make the output satable accros run
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ROOT = "."
chapter_id = "DNN"
image_path = os.path.join(ROOT,"images",chapter_id)
os.makedirs(image_path,exist_ok=True)

def save_fig(fig_id,tigh_layout=True,fig_extension="png",resolution=300):
    path = os.path.join(image_path,fig_id + "." + fig_extension)
    print("save figure ",fig_id)
    if tigh_layout:
        plt.tight_layout()
    plt.savefig(path,format=fig_extension,dpi=resolution)


8. Deep learning on CIFAR 10

# a
Exercise : Build a DNN with 20 hidden layers of 100 neurons each, use HE initialization and the elu activation function


In [5]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model_A = keras.models.Sequential()

model_A.add(keras.layers.Flatten(input_shape=[32,32,3]))
for _ in range(20):
    model_A.add(keras.layers.Dense(100,activation="elu",
                                   kernel_initializer = 'he_normal'))

# b

Exercise : using nadam optimization and early stopping, train the network on the CIFAR 10 dataset, can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60k with 32 x 32 pixel color images (50k for train and 10k for testing) with 10 classes, so u ill need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the models arsitekture or hyperparameter

In [6]:
# lets add output layer to the model
model_A.add(keras.layers.Dense(10,activation="softmax"))

Lets use a nadam optimizers with learning rate 5e-5, tried learning rates,1e-5,3e-5,1e-4,1e-3,3e-4, and 1e-2 and I compared their learning curves for 10 epochs each (using tensorboard callback, below) the learning rates 3e-5 and 1e-4 is prety good, so I tried 5e-5 which turned out to be slighly better

In [7]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5,beta_1=0.9,beta_2=0.999)
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])


Lets load the CIFAR10 dataset. We also want to use early stopping, so we need a validation set. Lets use 5k images from 50k train images set as the validation set now the train images only 45k

In [8]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.cifar10.load_data()
# comprased train data
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
# comprased validation data
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]



170498071/170498071 [==============================] - 866s 5us/step


In [17]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10.h5",save_best_only=True)
run_index = 1 # increment setiap melakukan train model
run_logdir = os.path.join(os.curdir,"my_cifar10_logs","run{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks=[early_stopping_cb, model_checkpoint_cb, tensorboard_cb]


In [18]:
%tensorboard --logdir =./my_cifar10_logs --port 6006

Reusing TensorBoard on port 6006 (pid 5160), started 0:29:38 ago. (Use '!kill 5160' to kill it.)

In [19]:
model_A.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.0856 - accuracy: 0.6095 - val_loss: 1.5794 - val_accuracy: 0.4756
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.0734 - accuracy: 0.6136 - val_loss: 1.5863 - val_accuracy: 0.4832
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.0681 - accuracy: 0.6159 - val_loss: 1.6167 - val_accuracy: 0.4814
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.0610 - accuracy: 0.6186 - val_loss: 1.5918 - val_accuracy: 0.4840
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.0568 - accuracy: 0.6200 - val_loss: 1.5856 - val_accuracy: 0.4764
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.0567 - accuracy: 0.6197 - val_loss: 1.6048 - val_accuracy: 0.4652
Epoch 7/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.0473 - accuracy: 0.6213 - val_loss: 1.6338 -

In [21]:
model_A = keras.models.load_model("cifar10.h5")
model_A.evaluate(X_valid,y_valid)

157/157 [==============================] - 1s 2ms/step - loss: 1.5794 - accuracy: 0.4756


[1.5793873071670532, 0.475600004196167]

the model with the lowest validation loss gets about 47.6% accuracy on the validation set.it took 21 epochs to reach the lowest validation loss, now lets see if we can iprove performance using batch normalization